# 🐍 Semgrep SAST Scanner Demo
**Globomantics Robot Fleet Manager** | Module 5 — Third-Party Scanners & SARIF Integration

This notebook demonstrates Semgrep as a third-party SAST scanner, SARIF output inspection, and integration with GitHub's unified Security tab.

In [1]:
import subprocess
import json
import os

REPO = "timothywarner-org/globomantics-robot-fleet"
REPO_DIR = r"C:\github\globomantics-robot-fleet"

# Ensure CodeQL CLI is in PATH for subprocess calls
os.environ["PATH"] = r"C:\codeql-home\codeql" + os.pathsep + os.environ["PATH"]
# Fix Semgrep Unicode encoding issue on Windows
os.environ["PYTHONUTF8"] = "1"

def run(cmd, cwd=REPO_DIR):
    """Run a shell command and display formatted output."""
    print(f"\u26a1 Running: {cmd}\n{'\u2500' * 60}")
    result = subprocess.run(
        cmd, shell=True, capture_output=True, text=True, cwd=cwd,
        env={**os.environ, "NO_COLOR": "1", "PYTHONUTF8": "1"}
    )
    if result.stdout.strip():
        print(result.stdout.strip())
    if result.stderr.strip():
        # Semgrep outputs progress to stderr, show it
        print(result.stderr.strip())
    print(f"{'\u2500' * 60}")
    return result

## 🔧 Pre-flight Checks

In [2]:
run("semgrep --version")
run("codeql --version")
run("gh auth status")

⚡ Running: semgrep --version
────────────────────────────────────────────────────────────


1.144.1
A new version of Semgrep is available. See https://semgrep.dev/docs/upgrading
────────────────────────────────────────────────────────────
⚡ Running: codeql --version
────────────────────────────────────────────────────────────


CodeQL command-line toolchain release 2.24.2.
Copyright (C) 2019-2026 GitHub, Inc.
Unpacked in: C:\codeql-home\codeql
   Analysis results depend critically on separately distributed query and
   extractor modules. To list modules that are visible to the toolchain,
   use 'codeql resolve packs' and 'codeql resolve languages'.
────────────────────────────────────────────────────────────
⚡ Running: gh auth status
────────────────────────────────────────────────────────────


github.com
  ✓ Logged in to github.com account timothywarner (keyring)
  - Active account: true
  - Git operations protocol: https
  - Token: gho_************************************
  - Token scopes: 'admin:org', 'admin:public_key', 'delete_repo', 'gist', 'repo', 'workflow'
────────────────────────────────────────────────────────────


CompletedProcess(args='gh auth status', returncode=0, stdout="\x1bgithub.com\x1b\n  \x1b✓\x1b Logged in to github.com account \x1btimothywarner\x1b (keyring)\n  - Active account: \x1btrue\x1b\n  - Git operations protocol: \x1bhttps\x1b\n  - Token: \x1bgho_************************************\x1b\n  - Token scopes: \x1b'admin:org', 'admin:public_key', 'delete_repo', 'gist', 'repo', 'workflow'\x1b\n", stderr='')

## 🔍 Full Repository Scan (JavaScript + Rust)

Two Semgrep config packs stacked:
- **`p/security-audit`** — covers JavaScript, general web patterns, and common CWEs
- **`p/rust`** — catches unsafe blocks, panics in library code, and Rust anti-patterns

Output is **SARIF 2.1.0**, the standard interchange format for static analysis results.
One scan command covers the entire polyglot repo.

In [3]:
import os
sarif_out = os.path.join(REPO_DIR, "results.sarif")
run(f'semgrep scan --config p/security-audit --config p/rust --sarif --output "{sarif_out}" .')
if os.path.exists(sarif_out):
    size_kb = os.path.getsize(sarif_out) / 1024
    print(f"\n\u2705 SARIF written: {sarif_out} ({size_kb:.1f} KB)")
else:
    print("\n\u274c SARIF file was not created")

⚡ Running: semgrep scan --config p/security-audit --config p/rust --sarif --output "C:\github\globomantics-robot-fleet\results.sarif" .
────────────────────────────────────────────────────────────


┌─────────────┐
│ Scan Status │
└─────────────┘
  Scanning 61 files tracked by git with 236 Code rules:
                                                                                                                        
  Language      Rules   Files          Origin      Rules                                                                
 ─────────────────────────────        ───────────────────                                                               
  <multilang>       2      61          Community     236                                                                
  yaml              1      10                                                                                           
  json              2       7                                                                                           
  python           78       4                                                                                           
  rust             11       4                    

## 🦀 Rust-Only Scan

Targeting just the `rust-telemetry-cli/` directory with Rust-specific rules.

**Why this matters:** CodeQL does not support Rust. Its supported languages are JavaScript,
Python, Java, C/C++, C#, Go, Ruby, Swift, and Kotlin. For Rust coverage, you need a
third-party scanner like Semgrep. This is exactly why GHAS supports SARIF uploads from
external tools.

In [4]:
import os
rust_sarif = os.path.join(REPO_DIR, "rust-scan.sarif")
run(f'semgrep scan --config p/rust --sarif --output "{rust_sarif}" ./rust-telemetry-cli')
if os.path.exists(rust_sarif):
    size_kb = os.path.getsize(rust_sarif) / 1024
    print(f"\n\u2705 SARIF written: {rust_sarif} ({size_kb:.1f} KB)")
else:
    print("\n\u274c SARIF file was not created")

⚡ Running: semgrep scan --config p/rust --sarif --output "C:\github\globomantics-robot-fleet\rust-scan.sarif" ./rust-telemetry-cli
────────────────────────────────────────────────────────────


┌─────────────┐
│ Scan Status │
└─────────────┘
  Scanning 8 files tracked by git with 11 Code rules:
                                                                                                                        
  Language   Rules   Files          Origin      Rules                                                                   
 ──────────────────────────        ───────────────────                                                                  
  rust          11       4          Community      11                                                                   
  json           1       1                                                                                              
                                                                                                                        
                
                
┌──────────────┐
│ Scan Summary │
└──────────────┘
✅ Scan completed successfully.
 • Findings: 1 (1 blocking)
 • Rules run: 11
 • Targets 

## 📊 SARIF Structure Inspection

SARIF 2.1.0 has a clean hierarchical structure:

```
sarif
├─ $schema / version
└─ runs[]
   ├─ tool.driver
   │  ├─ name, semanticVersion
   │  └─ rules[] (definitions of each check)
   └─ results[]
      ├─ ruleId
      ├─ message.text
      ├─ locations[].physicalLocation (file + line)
      └─ partialFingerprints (tracks alerts across commits)
```

**GH-500 Exam Tip:** `partialFingerprints` is how GitHub tracks alerts across commits —
even through file renames. The SARIF version **must be 2.1.0** for GitHub ingestion.

In [5]:
sarif_path = os.path.join(REPO_DIR, "results.sarif")
if os.path.exists(sarif_path):
    with open(sarif_path, 'r') as f:
        sarif = json.load(f)

    for i, run_data in enumerate(sarif.get("runs", [])):
        tool = run_data.get("tool", {}).get("driver", {})
        results = run_data.get("results", [])
        rules = tool.get("rules", [])

        print(f"\ud83d\udcca Run {i+1}")
        print(f"   \ud83d\udd27 Tool: {tool.get('name', 'unknown')} v{tool.get('semanticVersion', '?')}")
        print(f"   \ud83d\udccb Rules loaded: {len(rules)}")
        print(f"   \ud83c\udfaf Results found: {len(results)}")
        print(f"{'\u2500' * 60}")

        if results:
            print(f"\n\ud83d\udd0d First 10 findings:")
            for r in results[:10]:
                rule_id = r.get("ruleId", "?")
                msg = r.get("message", {}).get("text", "")[:80]
                locs = r.get("locations", [])
                if locs:
                    phys = locs[0].get("physicalLocation", {})
                    file = phys.get("artifactLocation", {}).get("uri", "?")
                    line = phys.get("region", {}).get("startLine", "?")
                    print(f"   \u26a0\ufe0f  {rule_id}")
                    print(f"      \ud83d\udcc1 {file}:{line}")
                    print(f"      \ud83d\udcac {msg}")
                    print()
else:
    print("\u274c results.sarif not found \u2014 run the full scan first (Cell 6)")

Exception in callback BaseAsyncIOLoop._handle_events()
handle: <Handle BaseAsyncIOLoop._handle_events()>
Traceback (most recent call last):
  File "C:\Users\timot\AppData\Roaming\Python\Python314\site-packages\jupyter_client\session.py", line 103, in json_packer
    ).encode("utf8", errors="surrogateescape")
      ~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 28-29: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Python314\Lib\asyncio\events.py", line 94, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\timot\AppData\Roaming\Python\Python314\site-packages\tornado\platform\asyncio.py", line 208, in _handle_events
    handler_func(fileobj, events)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "C:\Users\timot\AppData\Roaming\Python\Python314\site-packages\zmq\even

## 🚀 Upload SARIF to GitHub

The `codeql github upload-results` command bridges non-GitHub CI systems (Jenkins,
Azure DevOps, GitLab CI, CircleCI) with the GitHub Security tab.

The `--sarif-category` flag is **critical** — it prevents Semgrep results from
overwriting CodeQL results. Each scanner or upload source needs its own category.

**GH-500 Exam Tip:** Categories are required when using multiple scanners. Without them,
the latest upload silently replaces previous results.

In [6]:
# Get current commit hash
result = subprocess.run("git rev-parse HEAD", shell=True, capture_output=True, text=True, cwd=REPO_DIR)
commit = result.stdout.strip()
print(f"\ud83d\udccc Current commit: {commit}\n")

# \u26a0\ufe0f This uploads to GitHub \u2014 uncomment to execute
# run(f"codeql github upload-results --repository={REPO} --ref=refs/heads/main --commit={commit} --sarif=results.sarif --sarif-category=semgrep-local")

print(f"\ud83d\udd17 View results: https://github.com/{REPO}/security/code-scanning")

Exception in callback BaseAsyncIOLoop._handle_events()
handle: <Handle BaseAsyncIOLoop._handle_events()>
Traceback (most recent call last):
  File "C:\Users\timot\AppData\Roaming\Python\Python314\site-packages\jupyter_client\session.py", line 103, in json_packer
    ).encode("utf8", errors="surrogateescape")
      ~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 28-29: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Python314\Lib\asyncio\events.py", line 94, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\timot\AppData\Roaming\Python\Python314\site-packages\tornado\platform\asyncio.py", line 208, in _handle_events
    handler_func(fileobj, events)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "C:\Users\timot\AppData\Roaming\Python\Python314\site-packages\zmq\even

## 📋 View Alerts in GitHub

In [7]:
# All alerts
print("\ud83d\udccb All Code Scanning Alerts:")
print("=" * 60)
run(f'gh api repos/{REPO}/code-scanning/alerts --jq ".[] | {{number, rule: .rule.id, tool: .tool.name, state: .state}}"')

print("\n\ud83d\udc0d Semgrep OSS Alerts Only:")
print("=" * 60)
run(f'gh api repos/{REPO}/code-scanning/alerts --jq ".[] | select(.tool.name == \\"Semgrep OSS\\") | {{number, rule: .rule.id}}"')

{
"number": 7,
"rule": "javascript.express.security.audit.possible-user-input-redirect.unknown-value-in-redirect"
}
{
"number": 6,
"rule": "javascript.jsonwebtoken.security.jwt-hardcode.hardcoded-jwt-secret"
}
{
"number": 5,
"rule": "javascript.lang.security.detect-child-process.detect-child-process"
}
{
"number": 4,
"rule": "rust.lang.security.args.args"
}
────────────────────────────────────────────────────────────


CompletedProcess(args='gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.tool.name == \\"Semgrep OSS\\") | {number, rule: .rule.id}"', returncode=0, stdout='\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 7\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"javascript.express.security.audit.possible-user-input-redirect.unknown-value-in-redirect"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 6\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"javascript.jsonwebtoken.security.jwt-hardcode.hardcoded-jwt-secret"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 5\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"javascript.lang.security.detect-child-process.detect-child-process"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 4\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"rust.lang.security.args.args"\x1b[m\n\x1b}\x1b[m\n', stderr='')

## 💡 GH-500 Exam Quick Reference

| Topic | Key Point |
|-------|----------|
| SARIF upload action | `github/codeql-action/upload-sarif@v4` (NOT `codeql-action/analyze`) |
| Permission required | `security-events: write` — 403 without it |
| Categories | REQUIRED for multiple scanners — prevents result overwrites |
| Autofix scope | Works on **CodeQL alerts only**, not third-party results |
| SARIF version | Must be **2.1.0** |
| Fingerprints | `partialFingerprints` track alerts across commits |
| Semgrep coverage | 30+ languages including Rust (which CodeQL doesn't support) |